# Import Libraries

In [1]:
# Standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()


# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


# Regression algorithms
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import SGDRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor


# Comparing algorithms
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate


# Metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score


# Saving models
import pickle 


# Interpretability
import lime.lime_tabular

# Load data

In [2]:
df = pd.read_csv('train_set_preprocessed.csv')
df.head()

,price,year,condition,cylinders,fuel_gas,odometer,title_status_ok,transmission_automatic,manufacturer_asia,manufacturer_north america,...,type_pickup,type_sedan,type_truck,paint_color_black,paint_color_blue,paint_color_grey,paint_color_white,regions_usa_Midwest,regions_usa_South,regions_usa_West
0,46995.0,0.444444,0.235830,1.067818,0.0,-0.411426,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,36900.0,0.333333,0.270402,1.067818,0.0,-0.155110,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1399.0,0.888889,0.097933,-0.859197,0.0,-0.906978,0.0,0.0,0.0,-1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,29590.0,0.555556,-0.729598,0.000000,0.0,-0.515449,0.0,-1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,7950.0,-0.555556,0.270402,1.067818,0.0,0.875034,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [3]:
df.shape

(300999, 23)

# Preparing the data

In [4]:
X_train = df.iloc[:, 1:].values
y_train = df.iloc[:, 0].values

# First Round Modelling

## Setting things up
* **Metrics**
To assess performance on a regression task, a few common metrics are:
   *  **Mean Absolute Error (MAE)** 
       * Obtained by calculating the absolute difference between the model predictions and the true actual values: $\sum(\hat{y}-y)/N$. It is more robust to outliers but less efficient (mathematically/computation)
   * **Mean Square Error (MSE) / Root MSE (RMSE)**
       * Calculated by the sum of square of prediction error: $\sum(\hat{y}-y)^2/N$. Less robust to outliers, but more computation efficient.
   * **R2**
       * R2 is a measure of ...
       
    In this analysis, models will be compared on the RMSE metric.
* **Models**
    * Using the sklearn library, different regression models will be tested as benchmark (the best ones will be later fine-tuned)
* **Scores**
    * An empty list will be created to keep the scores of each metric, for each model

In [5]:
metrics = ['neg_root_mean_squared_error', 'neg_mean_absolute_error', 'r2']

In [6]:
models = []
scores = []

## Exploring models

In [7]:
%%time
sk_classes = [LinearRegression(), 
              ElasticNet(random_state = 42),
              KNeighborsRegressor(),
              DecisionTreeRegressor(random_state = 42),
              RandomForestRegressor(random_state = 42),
              AdaBoostRegressor(random_state = 42),
              GradientBoostingRegressor(random_state = 42)]


for model in sk_classes:
    model.fit(X_train, y_train)
    
    cross_val = cross_validate(model, X_train, y_train, scoring = metrics)

    models.append(f'{model.__class__.__name__}')
    scores.append(list(cross_val.values())[2:])

CPU times: total: 1h 26s
Wall time: 1h 19s


In [8]:
data = []
for i in range(len(models)):
    for j in range(len(metrics)):
        data.append(scores[i][j].mean())

data = np.reshape(data, (len(models), len(metrics)))
        
df_models = pd.DataFrame(data, index = models, columns = metrics)
df_models.style.highlight_max(color = 'lightgreen', axis = 0)

,neg_root_mean_squared_error,neg_mean_absolute_error,r2
LinearRegression,-8523.445444,-6178.590757,0.600677
ElasticNet,-9893.699099,-7537.524846,0.461979
KNeighborsRegressor,-4754.027172,-2715.579858,0.875764
DecisionTreeRegressor,-4556.838026,-1994.702462,0.885864
RandomForestRegressor,-3349.444586,-1628.561835,0.938324
AdaBoostRegressor,-8647.481058,-6896.778260,0.588824
GradientBoostingRegressor,-5737.981240,-4000.271365,0.819027


In [9]:
df_models[df_models.r2 > 0.8].style.highlight_max(color = 'lightgreen', axis = 0)

,neg_root_mean_squared_error,neg_mean_absolute_error,r2
KNeighborsRegressor,-4754.027172,-2715.579858,0.875764
DecisionTreeRegressor,-4556.838026,-1994.702462,0.885864
RandomForestRegressor,-3349.444586,-1628.561835,0.938324
GradientBoostingRegressor,-5737.981240,-4000.271365,0.819027


## Simplifying the model & comparing

In [10]:
rfr_gd_benchmark = RandomForestRegressor(random_state = 42)
rfr_gd_benchmark.fit(X_train, y_train)

df_model = pd.DataFrame(
{
    'features':df.iloc[:, 1:].columns,
    'importance': rfr_gd_benchmark.feature_importances_
})
df_model = df_model.sort_values(by = ['importance'], ascending = False)
df_model['cum_sum'] = df_model.importance.cumsum()

In [11]:
df_model

,features,importance,cum_sum
0,year,0.429553,0.429553
2,cylinders,0.249528,0.679080
4,odometer,0.129389,0.808469
1,condition,0.057065,0.865535
3,fuel_gas,0.043597,0.909132
8,manufacturer_north america,0.014433,0.923564
6,transmission_automatic,0.008321,0.931885
14,type_truck,0.007777,0.939663
10,drive_rwd,0.007567,0.947230
9,drive_fwd,0.007363,0.954593


### Feature selection

In [12]:
features_to_drop = df_model[df_model.cum_sum > 0.98].features
features_to_drop

18      paint_color_white
15      paint_color_black
5         title_status_ok
17       paint_color_grey
20      regions_usa_South
19    regions_usa_Midwest
16       paint_color_blue
Name: features, dtype: object

In [13]:
df_feature_selection = df.drop(features_to_drop, axis = 1)
df_feature_selection.shape

(300999, 16)

In [14]:
X_train = df_feature_selection.iloc[:, 1:].values
y_train = df_feature_selection.iloc[:, 0].values

In [15]:
models_2 = []
scores_2 = []

In [16]:
%%time
sk_classes = [LinearRegression(), 
               ElasticNet(random_state = 42),
               KNeighborsRegressor(),
               DecisionTreeRegressor(random_state = 42),
               RandomForestRegressor(random_state = 42),
               AdaBoostRegressor(random_state = 42),
               GradientBoostingRegressor(random_state = 42)]


for model in sk_classes:
    model.fit(X_train, y_train)
    
    cross_val = cross_validate(model, X_train, y_train, cv= 3, scoring = metrics)

    models_2.append(f'{model.__class__.__name__}')
    scores_2.append(list(cross_val.values())[2:])

CPU times: total: 9min 49s
Wall time: 9min 45s


In [17]:
data_2 = []
for i in range(len(models_2)):
    for j in range(len(metrics)):
        data_2.append(scores_2[i][j].mean())

data_2 = np.reshape(data_2, (len(models_2), len(metrics)))
        
df_models_2 = pd.DataFrame(data_2, index = models_2, columns = metrics)
df_models_2.style.highlight_max(color = 'lightgreen', axis = 0)

,neg_root_mean_squared_error,neg_mean_absolute_error,r2
LinearRegression,-8554.794103,-6205.670039,0.597744
ElasticNet,-9921.352337,-7564.368573,0.458975
KNeighborsRegressor,-4317.032513,-2369.183528,0.897562
DecisionTreeRegressor,-4650.231784,-2107.817703,0.881123
RandomForestRegressor,-3516.452022,-1757.286164,0.932030
AdaBoostRegressor,-8513.148113,-6744.030465,0.601654
GradientBoostingRegressor,-5736.153972,-3997.211922,0.819141


In [18]:
df_models_2[df_models_2.r2 > 0.8].style.highlight_max(color = 'lightgreen', axis = 0)

,neg_root_mean_squared_error,neg_mean_absolute_error,r2
KNeighborsRegressor,-4317.032513,-2369.183528,0.897562
DecisionTreeRegressor,-4650.231784,-2107.817703,0.881123
RandomForestRegressor,-3516.452022,-1757.286164,0.932030
GradientBoostingRegressor,-5736.153972,-3997.211922,0.819141


In [19]:
# For comparison
df_models[df_models.r2 > 0.8].style.highlight_max(color = 'lightgreen', axis = 0)

,neg_root_mean_squared_error,neg_mean_absolute_error,r2
KNeighborsRegressor,-4754.027172,-2715.579858,0.875764
DecisionTreeRegressor,-4556.838026,-1994.702462,0.885864
RandomForestRegressor,-3349.444586,-1628.561835,0.938324
GradientBoostingRegressor,-5737.981240,-4000.271365,0.819027


## Fine-Tune models

In [20]:
fine_tuned_models = []
new_models = []
new_scores = []

### RandomForestRegressor

In [21]:
%%time

'''RandomForestRegressor
parameters: 
    n_estimator: number of trees in the forest
    min_samples_split: min number of samples required to split an internal node
    max_features: n° features to consider for best split
    '''

param_grid = {'n_estimators':np.arange(200,350,25),
              'max_features':np.arange(0.2,0.8,0.2),
              'random_state':[42]}

rfr_gd = GridSearchCV(estimator = RandomForestRegressor(),
                      param_grid = param_grid, 
                      cv = 3, 
                      scoring = metrics,
                      refit = 'neg_root_mean_squared_error',
                      error_score = 'raise', 
                      verbose = 2)
                     
rfr_gd.fit(X_train, y_train)

Fitting 3 folds for each of 24 candidates, totalling 72 fits
[CV] END max_features=0.2, n_estimators=200, random_state=42; total time=  52.4s
[CV] END max_features=0.2, n_estimators=200, random_state=42; total time=  50.8s
[CV] END max_features=0.2, n_estimators=200, random_state=42; total time=  50.7s
[CV] END max_features=0.2, n_estimators=225, random_state=42; total time= 1.0min
[CV] END max_features=0.2, n_estimators=225, random_state=42; total time= 1.0min
[CV] END max_features=0.2, n_estimators=225, random_state=42; total time= 1.0min
[CV] END max_features=0.2, n_estimators=250, random_state=42; total time= 1.1min
[CV] END max_features=0.2, n_estimators=250, random_state=42; total time= 1.1min
[CV] END max_features=0.2, n_estimators=250, random_state=42; total time= 1.1min
[CV] END max_features=0.2, n_estimators=275, random_state=42; total time= 1.2min
[CV] END max_features=0.2, n_estimators=275, random_state=42; total time= 1.2min
[CV] END max_features=0.2, n_estimators=275, ran

GridSearchCV(cv=3, error_score='raise', estimator=RandomForestRegressor(),
             param_grid={'max_features': array([0.2, 0.4, 0.6, 0.8]),
                         'n_estimators': array([200, 225, 250, 275, 300, 325]),
                         'random_state': [42]},
             refit='neg_root_mean_squared_error',
             scoring=['neg_root_mean_squared_error', 'neg_mean_absolute_error',
                      'r2'],
             verbose=2)

In [22]:
fine_tuned_models.append(rfr_gd.best_estimator_)
rfr_gd.best_estimator_

RandomForestRegressor(max_features=0.4, n_estimators=325, random_state=42)

In [23]:
filename = 'RandomForestRegressor.sav'
pickle.dump(rfr_gd.best_estimator_, open(filename, 'wb'))

### KNeighborsRegressor

In [24]:
%%time

'''KNeighborsRegressor
parameters: 
    n_neihbors: number of neighbors to use by default
    weights: uniform / distance
    '''

param_grid = {'n_neighbors':np.arange(4,12,1),
              'weights':['uniform','distance']}

knr_gd = GridSearchCV(estimator = KNeighborsRegressor(),
                      param_grid = param_grid, 
                      cv = 3, 
                      scoring = metrics,
                      refit = 'neg_root_mean_squared_error',
                      error_score = 'raise', 
                      verbose = 2)
                     
knr_gd.fit(X_train, y_train)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV] END .....................n_neighbors=4, weights=uniform; total time=  31.6s
[CV] END .....................n_neighbors=4, weights=uniform; total time=  24.7s
[CV] END .....................n_neighbors=4, weights=uniform; total time=  24.0s
[CV] END ....................n_neighbors=4, weights=distance; total time=  24.3s
[CV] END ....................n_neighbors=4, weights=distance; total time=  24.3s
[CV] END ....................n_neighbors=4, weights=distance; total time=  24.2s
[CV] END .....................n_neighbors=5, weights=uniform; total time=  28.4s
[CV] END .....................n_neighbors=5, weights=uniform; total time=  28.7s
[CV] END .....................n_neighbors=5, weights=uniform; total time=  29.4s
[CV] END ....................n_neighbors=5, weights=distance; total time=  30.3s
[CV] END ....................n_neighbors=5, weights=distance; total time=  28.9s
[CV] END ....................n_neighbors=5, weig

GridSearchCV(cv=3, error_score='raise', estimator=KNeighborsRegressor(),
             param_grid={'n_neighbors': array([ 4,  5,  6,  7,  8,  9, 10, 11]),
                         'weights': ['uniform', 'distance']},
             refit='neg_root_mean_squared_error',
             scoring=['neg_root_mean_squared_error', 'neg_mean_absolute_error',
                      'r2'],
             verbose=2)

In [25]:
fine_tuned_models.append(knr_gd.best_estimator_)
knr_gd.best_estimator_

KNeighborsRegressor(n_neighbors=11, weights='distance')

In [26]:
filename = 'KNeighborsRegressor.sav'
pickle.dump(knr_gd.best_estimator_, open(filename, 'wb'))

### GradientBoostingRegressor

In [27]:
%%time

'''GradientBoostingRegressor
parameters: 
    learning_rate: it shrinks contribution of each tree by learning_rate
    n_estimator: number of boosting stages to perform (usually large number results in better performance)
    subsample: fraction of samples used for fitting, if < 1.0 => less variance (overfitting) , more bias (underfitting)
    max_features: n° features to consider for best split
    min_samples_split: min samples to split an internal node

    '''

param_grid = {'learning_rate':[0.15, 0.2, 0.25],
              'n_estimators':np.arange(200,350,25),
              'max_features':np.arange(0.2,1,0.2),
              'random_state':[42]}

gbr_gd = GridSearchCV(estimator = GradientBoostingRegressor(),
                      param_grid = param_grid, 
                      cv = 3, 
                      scoring = metrics,
                      refit = 'neg_root_mean_squared_error',
                      error_score = 'raise', 
                      verbose = 2)
                     
gbr_gd.fit(X_train, y_train)

Fitting 3 folds for each of 72 candidates, totalling 216 fits
[CV] END learning_rate=0.15, max_features=0.2, n_estimators=200, random_state=42; total time=  16.9s
[CV] END learning_rate=0.15, max_features=0.2, n_estimators=200, random_state=42; total time=  16.0s
[CV] END learning_rate=0.15, max_features=0.2, n_estimators=200, random_state=42; total time=  16.0s
[CV] END learning_rate=0.15, max_features=0.2, n_estimators=225, random_state=42; total time=  17.9s
[CV] END learning_rate=0.15, max_features=0.2, n_estimators=225, random_state=42; total time=  17.8s
[CV] END learning_rate=0.15, max_features=0.2, n_estimators=225, random_state=42; total time=  17.8s
[CV] END learning_rate=0.15, max_features=0.2, n_estimators=250, random_state=42; total time=  19.8s
[CV] END learning_rate=0.15, max_features=0.2, n_estimators=250, random_state=42; total time=  19.7s
[CV] END learning_rate=0.15, max_features=0.2, n_estimators=250, random_state=42; total time=  25.2s
[CV] END learning_rate=0.15, 

[CV] END learning_rate=0.2, max_features=0.2, n_estimators=250, random_state=42; total time=  20.0s
[CV] END learning_rate=0.2, max_features=0.2, n_estimators=250, random_state=42; total time=  20.4s
[CV] END learning_rate=0.2, max_features=0.2, n_estimators=250, random_state=42; total time=  25.0s
[CV] END learning_rate=0.2, max_features=0.2, n_estimators=275, random_state=42; total time=  22.5s
[CV] END learning_rate=0.2, max_features=0.2, n_estimators=275, random_state=42; total time=  21.5s
[CV] END learning_rate=0.2, max_features=0.2, n_estimators=275, random_state=42; total time=  21.3s
[CV] END learning_rate=0.2, max_features=0.2, n_estimators=300, random_state=42; total time=  24.1s
[CV] END learning_rate=0.2, max_features=0.2, n_estimators=300, random_state=42; total time=  23.3s
[CV] END learning_rate=0.2, max_features=0.2, n_estimators=300, random_state=42; total time=  23.3s
[CV] END learning_rate=0.2, max_features=0.2, n_estimators=325, random_state=42; total time=  25.8s


[CV] END learning_rate=0.25, max_features=0.2, n_estimators=300, random_state=42; total time=  31.8s
[CV] END learning_rate=0.25, max_features=0.2, n_estimators=325, random_state=42; total time=  35.9s
[CV] END learning_rate=0.25, max_features=0.2, n_estimators=325, random_state=42; total time=  36.0s
[CV] END learning_rate=0.25, max_features=0.2, n_estimators=325, random_state=42; total time=  35.0s
[CV] END learning_rate=0.25, max_features=0.4, n_estimators=200, random_state=42; total time=  36.5s
[CV] END learning_rate=0.25, max_features=0.4, n_estimators=200, random_state=42; total time=  37.4s
[CV] END learning_rate=0.25, max_features=0.4, n_estimators=200, random_state=42; total time=  36.4s
[CV] END learning_rate=0.25, max_features=0.4, n_estimators=225, random_state=42; total time=  41.6s
[CV] END learning_rate=0.25, max_features=0.4, n_estimators=225, random_state=42; total time=  43.4s
[CV] END learning_rate=0.25, max_features=0.4, n_estimators=225, random_state=42; total tim

GridSearchCV(cv=3, error_score='raise', estimator=GradientBoostingRegressor(),
             param_grid={'learning_rate': [0.15, 0.2, 0.25],
                         'max_features': array([0.2, 0.4, 0.6, 0.8]),
                         'n_estimators': array([200, 225, 250, 275, 300, 325]),
                         'random_state': [42]},
             refit='neg_root_mean_squared_error',
             scoring=['neg_root_mean_squared_error', 'neg_mean_absolute_error',
                      'r2'],
             verbose=2)

In [28]:
fine_tuned_models.append(gbr_gd.best_estimator_)
gbr_gd.best_estimator_

GradientBoostingRegressor(learning_rate=0.25, max_features=0.8,
                          n_estimators=325, random_state=42)

In [29]:
filename = 'GradientBoostingRegressor.sav'
pickle.dump(gbr_gd.best_estimator_, open(filename, 'wb'))

### Comparing models

In [30]:
%%time
for model in fine_tuned_models:
    cross_val = cross_validate(model, X_train, y_train, cv= 3, scoring = metrics)

    new_models.append(f'{model.__class__.__name__}')
    new_scores.append(list(cross_val.values())[2:])
    print('')

data = []
for model in range(len(fine_tuned_models)):
    for score in range(len(metrics)):
        data.append(new_scores[model][score].mean())

data = np.reshape(data, (len(fine_tuned_models), len(metrics)))
        
df_models = pd.DataFrame(data, index = new_models, columns = metrics)




CPU times: total: 14min 14s
Wall time: 14min 34s


In [31]:
df_models.style.highlight_max(color = 'lightgreen', axis = 0)

,neg_root_mean_squared_error,neg_mean_absolute_error,r2
RandomForestRegressor,-3390.631712,-1698.155388,0.936806
KNeighborsRegressor,-3813.632760,-1744.311863,0.920054
GradientBoostingRegressor,-4667.554135,-3150.937023,0.880248


# Final round of tuning

In [32]:
%%time

'''RandomForestRegressor
parameters: 
    n_estimator: number of trees in the forest
    min_samples_split: min number of samples required to split an internal node
    max_features: n° features to consider for best split
    '''

param_grid = {'n_estimators':np.arange(350,450,15),
              'max_features': [0.3, 0.4, 0.5],
              'random_state':[42]}

rfr_gd_2 = GridSearchCV(estimator = RandomForestRegressor(),
                      param_grid = param_grid, 
                      cv = 3, 
                      scoring = metrics,
                      refit = 'neg_root_mean_squared_error',
                      error_score = 'raise', 
                      verbose = 2)
                     
rfr_gd_2.fit(X_train, y_train)

Fitting 3 folds for each of 21 candidates, totalling 63 fits
[CV] END max_features=0.3, n_estimators=350, random_state=42; total time= 1.8min
[CV] END max_features=0.3, n_estimators=350, random_state=42; total time= 1.8min
[CV] END max_features=0.3, n_estimators=350, random_state=42; total time= 1.8min
[CV] END max_features=0.3, n_estimators=365, random_state=42; total time= 1.9min
[CV] END max_features=0.3, n_estimators=365, random_state=42; total time= 1.9min
[CV] END max_features=0.3, n_estimators=365, random_state=42; total time= 2.0min
[CV] END max_features=0.3, n_estimators=380, random_state=42; total time= 2.1min
[CV] END max_features=0.3, n_estimators=380, random_state=42; total time= 2.1min
[CV] END max_features=0.3, n_estimators=380, random_state=42; total time= 2.1min
[CV] END max_features=0.3, n_estimators=395, random_state=42; total time= 2.0min
[CV] END max_features=0.3, n_estimators=395, random_state=42; total time= 2.0min
[CV] END max_features=0.3, n_estimators=395, ran

GridSearchCV(cv=3, error_score='raise', estimator=RandomForestRegressor(),
             param_grid={'max_features': [0.3, 0.4, 0.5],
                         'n_estimators': array([350, 365, 380, 395, 410, 425, 440]),
                         'random_state': [42]},
             refit='neg_root_mean_squared_error',
             scoring=['neg_root_mean_squared_error', 'neg_mean_absolute_error',
                      'r2'],
             verbose=2)

In [33]:
rfr_gd_2.best_estimator_

RandomForestRegressor(max_features=0.4, n_estimators=425, random_state=42)

In [34]:
filename = 'RandomForestRegressor_2.sav'
pickle.dump(rfr_gd_2.best_estimator_, open(filename, 'wb'))

In [35]:
scores_ = cross_validate(rfr_gd_2.best_estimator_, X_train, y_train, cv= 3, scoring = metrics)

In [36]:
scores_.get('test_neg_root_mean_squared_error').mean()

-3388.411560918494

In [37]:
mean_squared_error_2 = round(scores_.get('test_neg_root_mean_squared_error').mean(), 2)
mean_absolute_error_2 = round(scores_.get('test_neg_mean_absolute_error').mean(),2)
r2_2 = round(scores_.get('test_r2').mean(),2)

In [38]:
print('neg_root_mean_squared_error:  ', mean_squared_error_2)
print('neg_mean_absolute_error:      ', mean_absolute_error_2)
print('r2:                            ', r2_2)

neg_root_mean_squared_error:   -3388.41
neg_mean_absolute_error:       -1696.43
r2:                             0.94


In [39]:
df_models.iloc[0]

neg_root_mean_squared_error   -3390.631712
neg_mean_absolute_error       -1698.155388
r2                                0.936806
Name: RandomForestRegressor, dtype: float64